## 2. PyTorch Implementation of Generative Adverarial Network (GAN) (50 points)

You are required to implement a simple Generative Adversarial Network (GAN) model and train it on the MNIST dataset. We have provided the main structure of the code in this Jupyter notebook, and you need to complete the code according to the comments.

In [1]:
# prerequisites
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.autograd import Variable


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Current device: {device}')

Current device: cuda


In [ ]:
# Do not adjust any hyper parameter!
batch_size = 128
lr = 0.0001
z_dim = 100

In [2]:
# MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))
])

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transform, download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
# Do not modify the model architecture of `Generator`
class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()       
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 1024)
        self.fc4 = nn.Linear(1024, g_output_dim)
    
    def forward(self, x): 
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = torch.tanh(self.fc4(x))
        
        return x


# Do not modify the model architecture of `Discriminator` 
class Discriminator(nn.Module):
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)
    
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = torch.sigmoid(self.fc4(x))
        
        return x

In [4]:
# build network
mnist_dim = train_dataset.train_data.size(1) * train_dataset.train_data.size(2)

G = Generator(g_input_dim=z_dim, g_output_dim=mnist_dim).to(device)
D = Discriminator(mnist_dim).to(device)

/home/zhoujk/anaconda3/envs/general/lib/python3.8/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [11]:
print(G)
print(D)

Generator(
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=784, bias=True)
)
Discriminator(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
)


In [7]:
# loss
loss_fn = nn.BCELoss() 

# optimizer
G_optimizer = torch.optim.Adam(G.parameters(), lr = lr)
D_optimizer = torch.optim.Adam(D.parameters(), lr = lr)

In [1]:
def D_train(x, G, D, mnist_dim, batch_size, z_dim, device):
    #=======================Train the discriminator=======================#
    D.zero_grad()

    # train discriminator on real
    x_real, y_real = x.view(-1, mnist_dim), torch.ones(batch_size, 1)
    x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))

    D_output = D(x_real)
    D_real_loss = loss_fn(D_output, y_real)

    # train discriminator on fake
    z = Variable(torch.randn(batch_size, z_dim).to(device))
    x_fake, y_fake = G(z), Variable(torch.zeros(batch_size, 1).to(device))

    D_output = D(x_fake)
    D_fake_loss = loss_fn(D_output, y_fake)

    # gradient backprop & optimize ONLY D's parameters
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()
        
    return  D_loss.data.item(), G, D


def G_train(G, D, batch_size, z_dim, device):
    #=======================Train the generator=======================#
    ################################################################################
    # TODO (30 points):                                                                        #
    # finish the procedure of training the generator                               #
    ################################################################################
    pass

In [ ]:
n_epoch = 100
for epoch in range(1, n_epoch+1):           
    D_losses, G_losses = [], []
    for batch_idx, (x, _) in enumerate(train_loader):
        D_loss, G, D = D_train(x, G, D, mnist_dim, batch_size, z_dim, device)
        G_loss, G, D = G_train(G, D, batch_size, z_dim, device)
        D_losses.append(D_loss)
        G_losses.append(G_loss)

    print(
        f'[{epoch}/{n_epoch}]: '
        f'loss_d: {torch.tensor(D_losses).mean():.3f}, '
        f'loss_g: {torch.tensor(G_losses).mean():.3f}'
    )

In [ ]:
################################################################################
# TODO (10 points):                                                                        #
# you need to plot the training loss of generator and discriminator            #
# you may need to modify the training code to record the corresponding loss    #
################################################################################

In [10]:
################################################################################
# TODO (10 points):                                                                        #
# you need to generate an image with the trained generator                     #
# a sample result is given as follows                                      #
################################################################################
